# Probabilistic Bayesian Neural Networks
https://keras.io/examples/keras_recipes/bayesian_neural_networks/

## Setup

### Ambiente
Creamos un ambiente con la paquetería necesaria

``conda create -n env_tf_bayes``

``conda activate env_tf_bayes``

``pip install tensorflow-probability``

``pip install tensorflow-datasets``

### Bibliotecas

In [24]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
import pandas as pd

## Create training and evaluation datasets

In [25]:
############# Código original
# def get_train_and_test_splits(train_size, batch_size=1):
#     # We prefetch with a buffer the same size as the dataset because th dataset
#     # is very small and fits into memory.
#     dataset = (
#         tfds.load(name="wine_quality", as_supervised=True, split="train")
#         .map(lambda x, y: (x, tf.cast(y, tf.float32)))
#         .prefetch(buffer_size=dataset_size)
#         .cache()
#     )
#     # We shuffle with a buffer the same size as the dataset.
#     train_dataset = (
#         dataset.take(train_size).shuffle(buffer_size=train_size).batch(batch_size)
#     )
#     test_dataset = dataset.skip(train_size).batch(batch_size)

#     return train_dataset, test_dataset

In [26]:
def get_train_and_test_splits(train_size, batch_size=1):
    # Importar datos
    data = pd.read_csv("./../data/train_wine.csv")

    # VARIABLE OBJETIVO Y VARIABLES INDEPENDIENTES
    features = data.drop(columns=["quality", "id"])
    labels = data["quality"]

    # Convertir a tensores de TensorFlow
    features_dict = {col: tf.convert_to_tensor(features[col].values, dtype=tf.float32) for col in features.columns}
    labels_tensor = tf.convert_to_tensor(labels.values, dtype=tf.float32)

    # Crear dataset de TensorFlow
    dataset = tf.data.Dataset.from_tensor_slices((features_dict, labels_tensor))
    dataset = dataset.cache().shuffle(len(data)).prefetch(buffer_size=tf.data.AUTOTUNE)

    # Definir train_size y test_size correctamente
    test_size = len(data) - train_size  # Corregir tamaño del dataset de prueba

    train_dataset = dataset.take(train_size).batch(batch_size)
    test_dataset = dataset.skip(train_size).take(test_size).batch(batch_size)  # Agregar `take(test_size)`

    return train_dataset, test_dataset

## Compile, train, and evaluate the model

In [27]:
hidden_units = [8, 8]
learning_rate = 0.001


def run_experiment(model, loss, train_dataset, test_dataset):

    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
        loss=loss,
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    print("Start training the model...")
    model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)
    print("Model training finished.")
    _, rmse = model.evaluate(train_dataset, verbose=0)
    print(f"Train RMSE: {round(rmse, 3)}")

    print("Evaluating model performance...")
    _, rmse = model.evaluate(test_dataset, verbose=0)
    print(f"Test RMSE: {round(rmse, 3)}")


## Create model inputs

In [28]:
FEATURE_NAMES = [
    "fixed acidity",
    "volatile acidity",
    "citric acid",
    "residual sugar",
    "chlorides",
    "free sulfur dioxide",
    "total sulfur dioxide",
    "density",
    "pH",
    "sulphates",
    "alcohol",
]


def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        inputs[feature_name] = layers.Input(
            name=feature_name, shape=(1,), dtype=tf.float32
        )
    return inputs


## Experiment 1: standard neural network
We create a standard deterministic neural network model as a baseline.

In [29]:

def create_baseline_model():
    inputs = create_model_inputs()
    input_values = [value for _, value in sorted(inputs.items())]
    features = keras.layers.concatenate(input_values)
    features = layers.BatchNormalization()(features)

    # Create hidden layers with deterministic weights using the Dense layer.
    for units in hidden_units:
        features = layers.Dense(units, activation="sigmoid")(features)
    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


Let's split the wine dataset into training and test sets, with 85% and 15% of the examples, respectively.

In [30]:
data = pd.read_csv("./../data/train_wine.csv")
dataset_size = data.shape[0]
batch_size = 256
train_size = int(dataset_size * 0.85)
train_dataset, test_dataset = get_train_and_test_splits(train_size, batch_size)

Now let's train the baseline model. We use the MeanSquaredError as the loss function.

In [31]:
num_epochs = 100
mse_loss = keras.losses.MeanSquaredError()
baseline_model = create_baseline_model()
run_experiment(baseline_model, mse_loss, train_dataset, test_dataset)

Start training the model...
Epoch 1/100


2025-04-13 18:59:07.149393: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [15000]
	 [[{{node Placeholder/_1}}]]
2025-04-13 18:59:07.149921: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_9' with dtype float and shape [15000]
	 [[{{node Placeholder/_9}}]]


41/50 [=======================>......] - ETA: 0s - loss: 39.6454 - root_mean_squared_error: 6.2965

2025-04-13 18:59:07.940151: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_9' with dtype float and shape [15000]
	 [[{{node Placeholder/_9}}]]
2025-04-13 18:59:07.944342: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_9' with dtype float and shape [15000]
	 [[{{node Placeholder/_9}}]]


50/50 [==============================] - 1s 13ms/step - loss: 39.1825 - root_mean_squared_error: 6.2596 - val_loss: 37.6696 - val_root_mean_squared_error: 6.1376
Epoch 2/100
50/50 [==============================] - 0s 8ms/step - loss: 34.3432 - root_mean_squared_error: 5.8603 - val_loss: 32.9744 - val_root_mean_squared_error: 5.7423
Epoch 3/100
50/50 [==============================] - 1s 9ms/step - loss: 30.1220 - root_mean_squared_error: 5.4884 - val_loss: 28.7054 - val_root_mean_squared_error: 5.3577
Epoch 4/100
50/50 [==============================] - 1s 10ms/step - loss: 26.0849 - root_mean_squared_error: 5.1073 - val_loss: 24.1416 - val_root_mean_squared_error: 4.9134
Epoch 5/100
50/50 [==============================] - 1s 9ms/step - loss: 22.3391 - root_mean_squared_error: 4.7264 - val_loss: 20.4257 - val_root_mean_squared_error: 4.5195
Epoch 6/100
50/50 [==============================] - 1s 10ms/step - loss: 19.0008 - root_mean_squared_error: 4.3590 - val_loss: 17.5182 - val_roo

We take a sample from the test set use the model to obtain predictions for them. Note that since the baseline model is deterministic, we get a single a point estimate prediction for each test example, with no information about the uncertainty of the model nor the prediction.

In [32]:
sample = 10
examples, targets = list(test_dataset.unbatch().shuffle(batch_size * 10).batch(sample))[
    0
]

predicted = baseline_model(examples).numpy()
for idx in range(sample):
    print(f"Predicted: {round(float(predicted[idx][0]), 1)} - Actual: {targets[idx]}")

2025-04-13 19:00:10.060081: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_6' with dtype float and shape [15000]
	 [[{{node Placeholder/_6}}]]
2025-04-13 19:00:10.061190: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_5' with dtype float and shape [15000]
	 [[{{node Placeholder/_5}}]]


Predicted: 5.4 - Actual: 5.0
Predicted: 5.4 - Actual: 7.0
Predicted: 5.6 - Actual: 6.0
Predicted: 5.3 - Actual: 5.0
Predicted: 6.5 - Actual: 5.0
Predicted: 5.4 - Actual: 5.0
Predicted: 5.6 - Actual: 6.0
Predicted: 5.4 - Actual: 6.0
Predicted: 5.4 - Actual: 6.0
Predicted: 6.5 - Actual: 7.0


## Experiment 2: Bayesian neural network (BNN)

The object of the Bayesian approach for modeling neural networks is to capture the epistemic uncertainty, which is uncertainty about the model fitness, due to limited training data.

The idea is that, instead of learning specific weight (and bias) values in the neural network, the Bayesian approach learns weight distributions - from which we can sample to produce an output for a given input - to encode weight uncertainty.

Thus, we need to define prior and the posterior distributions of these weights, and the training process is to learn the parameters of these distributions.

In [33]:
# Define the prior weight distribution as Normal of mean=0 and stddev=1.
# Note that, in this example, the we prior distribution is not trainable,
# as we fix its parameters.
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = keras.Sequential(
        [
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.MultivariateNormalDiag(
                    loc=tf.zeros(n), scale_diag=tf.ones(n)
                )
            )
        ]
    )
    return prior_model


# Define variational posterior weight distribution as multivariate Gaussian.
# Note that the learnable parameters for this distribution are the means,
# variances, and covariances.
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential(
        [
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ),
            tfp.layers.MultivariateNormalTriL(n),
        ]
    )
    return posterior_model


We use the tfp.layers.DenseVariational layer instead of the standard keras.layers.Dense layer in the neural network model.

In [34]:

def create_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


The epistemic uncertainty can be reduced as we increase the size of the training data. That is, the more data the BNN model sees, the more it is certain about its estimates for the weights (distribution parameters). Let's test this behaviour by training the BNN model on a small subset of the training set, and then on the full training set, to compare the output variances.

### Train BNN with a small training subset.

In [35]:
num_epochs = 500
train_sample_size = int(train_size * 0.3)
small_train_dataset = train_dataset.unbatch().take(train_sample_size).batch(batch_size)

bnn_model_small = create_bnn_model(train_sample_size)
run_experiment(bnn_model_small, mse_loss, small_train_dataset, test_dataset)

Start training the model...
Epoch 1/500


2025-04-13 19:17:02.992482: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype float and shape [15000]
	 [[{{node Placeholder/_2}}]]
2025-04-13 19:17:02.992905: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype float and shape [15000]
	 [[{{node Placeholder/_3}}]]


15/15 [==============================] - 2s 51ms/step - loss: 20.4911 - root_mean_squared_error: 4.5262 - val_loss: 16.5074 - val_root_mean_squared_error: 4.0623
Epoch 2/500
15/15 [==============================] - 0s 27ms/step - loss: 19.1516 - root_mean_squared_error: 4.3757 - val_loss: 18.0770 - val_root_mean_squared_error: 4.2511
Epoch 3/500
15/15 [==============================] - 1s 32ms/step - loss: 17.2203 - root_mean_squared_error: 4.1491 - val_loss: 17.0530 - val_root_mean_squared_error: 4.1289
Epoch 4/500
15/15 [==============================] - 0s 26ms/step - loss: 16.1263 - root_mean_squared_error: 4.0151 - val_loss: 14.9187 - val_root_mean_squared_error: 3.8618
Epoch 5/500
15/15 [==============================] - 0s 23ms/step - loss: 17.0107 - root_mean_squared_error: 4.1238 - val_loss: 14.6530 - val_root_mean_squared_error: 3.8273
Epoch 6/500
15/15 [==============================] - 0s 24ms/step - loss: 15.4667 - root_mean_squared_error: 3.9322 - val_loss: 13.6854 - val_

Since we have trained a BNN model, the model produces a different output each time we call it with the same input, since each time a new set of weights are sampled from the distributions to construct the network and produce an output. The less certain the mode weights are, the more variability (wider range) we will see in the outputs of the same inputs.

In [36]:

def compute_predictions(model, iterations=100):
    predicted = []
    for _ in range(iterations):
        predicted.append(model(examples).numpy())
    predicted = np.concatenate(predicted, axis=1)

    prediction_mean = np.mean(predicted, axis=1).tolist()
    prediction_min = np.min(predicted, axis=1).tolist()
    prediction_max = np.max(predicted, axis=1).tolist()
    prediction_range = (np.max(predicted, axis=1) - np.min(predicted, axis=1)).tolist()

    for idx in range(sample):
        print(
            f"Predictions mean: {round(prediction_mean[idx], 2)}, "
            f"min: {round(prediction_min[idx], 2)}, "
            f"max: {round(prediction_max[idx], 2)}, "
            f"range: {round(prediction_range[idx], 2)} - "
            f"Actual: {targets[idx]}"
        )


compute_predictions(bnn_model_small)

Predictions mean: 5.37, min: 4.87, max: 5.84, range: 0.97 - Actual: 5.0
Predictions mean: 5.34, min: 4.86, max: 5.83, range: 0.98 - Actual: 7.0
Predictions mean: 5.61, min: 5.15, max: 5.93, range: 0.78 - Actual: 6.0
Predictions mean: 5.24, min: 4.7, max: 5.76, range: 1.05 - Actual: 5.0
Predictions mean: 6.23, min: 5.89, max: 6.37, range: 0.47 - Actual: 5.0
Predictions mean: 5.3, min: 4.81, max: 5.79, range: 0.99 - Actual: 5.0
Predictions mean: 5.6, min: 5.19, max: 5.96, range: 0.77 - Actual: 6.0
Predictions mean: 5.38, min: 4.92, max: 5.85, range: 0.93 - Actual: 6.0
Predictions mean: 5.35, min: 4.89, max: 5.84, range: 0.94 - Actual: 6.0
Predictions mean: 6.22, min: 5.88, max: 6.36, range: 0.49 - Actual: 7.0


### Train BNN with the whole training set.

In [37]:
num_epochs = 500
bnn_model_full = create_bnn_model(train_size)
run_experiment(bnn_model_full, mse_loss, train_dataset, test_dataset)

compute_predictions(bnn_model_full)

Start training the model...
Epoch 1/500
50/50 [==============================] - 2s 18ms/step - loss: 39.6777 - root_mean_squared_error: 6.2989 - val_loss: 37.5666 - val_root_mean_squared_error: 6.1291
Epoch 2/500
50/50 [==============================] - 1s 12ms/step - loss: 34.6732 - root_mean_squared_error: 5.8883 - val_loss: 34.8258 - val_root_mean_squared_error: 5.9012
Epoch 3/500
50/50 [==============================] - 1s 12ms/step - loss: 30.9026 - root_mean_squared_error: 5.5589 - val_loss: 28.0429 - val_root_mean_squared_error: 5.2954
Epoch 4/500
50/50 [==============================] - 1s 10ms/step - loss: 26.9265 - root_mean_squared_error: 5.1889 - val_loss: 26.4711 - val_root_mean_squared_error: 5.1449
Epoch 5/500
50/50 [==============================] - 1s 12ms/step - loss: 24.0908 - root_mean_squared_error: 4.9081 - val_loss: 22.8611 - val_root_mean_squared_error: 4.7812
Epoch 6/500
50/50 [==============================] - 1s 11ms/step - loss: 21.8903 - root_mean_squared_

## Experiment 3: probabilistic Bayesian neural network
So far, the output of the standard and the Bayesian NN models that we built is deterministic, that is, produces a point estimate as a prediction for a given example. We can create a probabilistic NN by letting the model output a distribution. In this case, the model captures the aleatoric uncertainty as well, which is due to irreducible noise in the data, or to the stochastic nature of the process generating the data.

In this example, we model the output as a IndependentNormal distribution, with learnable mean and variance parameters. If the task was classification, we would have used IndependentBernoulli with binary classes, and OneHotCategorical with multiple classes, to model distribution of the model output.

In [38]:

def create_probablistic_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # Create a probabilisticå output (Normal distribution), and use the `Dense` layer
    # to produce the parameters of the distribution.
    # We set units=2 to learn both the mean and the variance of the Normal distribution.
    distribution_params = layers.Dense(units=2)(features)
    outputs = tfp.layers.IndependentNormal(1)(distribution_params)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


Since the output of the model is a distribution, rather than a point estimate, we use the negative loglikelihood as our loss function to compute how likely to see the true data (targets) from the estimated distribution produced by the model.

In [39]:

def negative_loglikelihood(targets, estimated_distribution):
    return -estimated_distribution.log_prob(targets)


num_epochs = 1000
prob_bnn_model = create_probablistic_bnn_model(train_size)
run_experiment(prob_bnn_model, negative_loglikelihood, train_dataset, test_dataset)

Start training the model...
Epoch 1/1000
50/50 [==============================] - 2s 21ms/step - loss: 28.2965 - root_mean_squared_error: 5.5721 - val_loss: 23.0112 - val_root_mean_squared_error: 5.4127
Epoch 2/1000
50/50 [==============================] - 0s 9ms/step - loss: 18.1836 - root_mean_squared_error: 5.3397 - val_loss: 15.7517 - val_root_mean_squared_error: 5.3271
Epoch 3/1000
50/50 [==============================] - 1s 12ms/step - loss: 11.4211 - root_mean_squared_error: 5.0994 - val_loss: 11.9173 - val_root_mean_squared_error: 5.0302
Epoch 4/1000
50/50 [==============================] - 1s 12ms/step - loss: 7.5697 - root_mean_squared_error: 4.9731 - val_loss: 5.4789 - val_root_mean_squared_error: 4.7786
Epoch 5/1000
50/50 [==============================] - 1s 12ms/step - loss: 5.4876 - root_mean_squared_error: 4.7777 - val_loss: 4.4158 - val_root_mean_squared_error: 4.7437
Epoch 6/1000
50/50 [==============================] - 1s 10ms/step - loss: 4.1444 - root_mean_squared_

Now let's produce an output from the model given the test examples. The output is now a distribution, and we can use its mean and variance to compute the confidence intervals (CI) of the prediction.

In [40]:
prediction_distribution = prob_bnn_model(examples)
prediction_mean = prediction_distribution.mean().numpy().tolist()
prediction_stdv = prediction_distribution.stddev().numpy()

# The 95% CI is computed as mean ± (1.96 * stdv)
upper = (prediction_mean + (1.96 * prediction_stdv)).tolist()
lower = (prediction_mean - (1.96 * prediction_stdv)).tolist()
prediction_stdv = prediction_stdv.tolist()

for idx in range(sample):
    print(
        f"Prediction mean: {round(prediction_mean[idx][0], 2)}, "
        f"stddev: {round(prediction_stdv[idx][0], 2)}, "
        f"95% CI: [{round(upper[idx][0], 2)} - {round(lower[idx][0], 2)}]"
        f" - Actual: {targets[idx]}"
    )

Prediction mean: 5.38, stddev: 0.64, 95% CI: [6.65 - 4.12] - Actual: 5.0
Prediction mean: 5.35, stddev: 0.62, 95% CI: [6.56 - 4.14] - Actual: 7.0
Prediction mean: 5.54, stddev: 0.71, 95% CI: [6.93 - 4.14] - Actual: 6.0
Prediction mean: 5.3, stddev: 0.59, 95% CI: [6.45 - 4.15] - Actual: 5.0
Prediction mean: 6.4, stddev: 0.81, 95% CI: [7.99 - 4.8] - Actual: 5.0
Prediction mean: 5.36, stddev: 0.62, 95% CI: [6.59 - 4.14] - Actual: 5.0
Prediction mean: 5.54, stddev: 0.73, 95% CI: [6.98 - 4.1] - Actual: 6.0
Prediction mean: 5.37, stddev: 0.63, 95% CI: [6.61 - 4.13] - Actual: 6.0
Prediction mean: 5.37, stddev: 0.63, 95% CI: [6.6 - 4.13] - Actual: 6.0
Prediction mean: 6.4, stddev: 0.81, 95% CI: [8.0 - 4.8] - Actual: 7.0
